<h1> Data Analysis for REYeker</h1>

In [1]:
# lib for dataframes
import pandas as pd

# lib for saving np images
from PIL import Image

# lib for plotting
#%matplotlib inline
%matplotlib qt
import matplotlib.pyplot as plt

# lib for numerical computations
import numpy as np

import seaborn as sns

# lib for crerating paths
from pathlib import Path

# REYeker lib
import modules.rEYEkerAnalysis as rEYEker

<h2>1. Configuration</h2>

<h5>Database configuration </h5>

In [2]:
# path to the datafile
config_datasheet_path_pre = r'./results/data_of_all_removed.xlsx'
config_datasheet_path_post = r'./results/data_of_all_removed_posttest.xlsx'
compare_datasheet = "results/behavioral/compare/compare.xlsx"

# columns with visual stimulus data
config_visual_stimulus_variable = "ClickData"

# columns with names of the algo
config_algo_name_variable = "Algorithm"

# columns with correctness value
config_corectness_variable = "Correctness"

# colums of response time
config_response_time_variable = "ResponseTime"

<h5>Configuration for REYEker data </h5>

In [3]:
# file for loading rEYEker settings
config_reyeker_settings_path = "data/used.json"

<h5>Import the preprocessed dataframe</h5>

In [4]:
df_compare = pd.read_excel(compare_datasheet)

df_pre_tmp = pd.read_excel(config_datasheet_path_pre)
df_post_tmp = pd.read_excel(config_datasheet_path_post)


df_pre= pd.DataFrame()
df_post = pd.DataFrame()

df_pre_correct = pd.DataFrame()
df_post_correct = pd.DataFrame()

df_pre_false = pd.DataFrame()
df_post_false = pd.DataFrame()

unique_codes = df_compare["Unique Code"].unique()

for code in unique_codes:
    tmp_pre_correct = df_pre_tmp.loc[(df_pre_tmp["Unique Code"] == code) & (df_pre_tmp["Correctness"] == True)]
    tmp_pre_false = df_pre_tmp.loc[(df_pre_tmp["Unique Code"] == code) & (df_pre_tmp["Correctness"] == False)]
    tmp_pre = df_pre_tmp.loc[df_pre_tmp["Unique Code"]== code]
    
    tmp_post_correct = df_post_tmp.loc[(df_post_tmp["Unique Code"] == code) & (df_post_tmp["Correctness"] == True)]
    tmp_post_false = df_post_tmp.loc[(df_post_tmp["Unique Code"] == code) & (df_post_tmp["Correctness"] == False)]
    tmp_post = df_post_tmp.loc[df_post_tmp["Unique Code"]== code]
    
    df_pre_correct = df_pre_correct.append(tmp_pre_correct)
    df_pre_false = df_pre_false.append(tmp_pre_false)
    df_pre = df_pre.append(tmp_pre)

    df_post_correct = df_post_correct.append(tmp_post_correct)
    df_post_false = df_post_false.append(tmp_post_false)
    df_post = df_post.append(tmp_post)

print(len(df_pre))
print(len(df_post))
print(len(df_pre_correct), len(df_pre_false), len(df_pre))
print(len(df_post_correct), len(df_post_false), len(df_post))

209
209
176 33 209
168 41 209


In [5]:
algo_name_array_pre = df_pre["Algorithm"].unique()
algo_name_array_post = df_post["Algorithm"].unique()

df_array_pre = []
df_array_pre_false = []
df_array_pre_correct = []

df_array_post = []
df_array_post_false = []
df_array_post_correct = []

for algo_name in algo_name_array_pre:
    algo_df = df_pre.loc[df_pre[config_algo_name_variable]==algo_name]
    algo_df_false = df_pre_false.loc[df_pre_false[config_algo_name_variable]==algo_name]
    algo_df_correct = df_pre_correct.loc[df_pre_correct[config_algo_name_variable]==algo_name]
    
    
    df_array_pre.append(algo_df)
    df_array_pre_false.append(algo_df_false)
    df_array_pre_correct.append(algo_df_correct)


for algo_name in algo_name_array_post:
    algo_df = df_post.loc[df_post[config_algo_name_variable]==algo_name]
    algo_df_false = df_post_false.loc[df_post_false[config_algo_name_variable]==algo_name]
    algo_df_correct = df_post_correct.loc[df_post_correct[config_algo_name_variable]==algo_name]
    
    df_array_post.append(algo_df)
    df_array_post_false.append(algo_df_false)
    df_array_post_correct.append(algo_df_correct)


In [6]:
# data for loading the images
image_path_array_pre = []
image_path_array_post = []

for algo_name in algo_name_array_pre:
    image_path = 'images/Pretest/' + algo_name + '.png'
    image_path_array_pre.append(image_path)
    
for algo_name in algo_name_array_post:
    image_path = 'images/Posttest/' + algo_name + '.png'
    image_path_array_post.append(image_path)
    
print(image_path_array_post)

['images/Posttest/CommonChars.png', 'images/Posttest/IsPalindrome.png', 'images/Posttest/ReverseString.png', 'images/Posttest/ReverseArray.png', 'images/Posttest/BinarySearchStrings.png', 'images/Posttest/Multiples.png', 'images/Posttest/Power.png', 'images/Posttest/GetMiddle.png', 'images/Posttest/InsertionSort.png', 'images/Posttest/SquareRoot.png']


<h4>Import REYeker Settings</h4>

In [7]:
(_data, _times, click_setting) = rEYEker.load_data_from_json(config_reyeker_settings_path)

<h4>Import Images Settings</h4>

In [8]:
image_array_pre = []
image_array_post = []

# read in every image
for image_path in image_path_array_pre:
    image = rEYEker.load_image(image_path)
   
    image_array_pre.append(image)
    
    
for image_path in image_path_array_post:
    image = rEYEker.load_image(image_path)
    
    image_array_post.append(image)


<h4> Cast Data to Valid format</h4>

Import the visual stimulus measured Data

correct and false

In [9]:
visual_stimulus_data_matrix_pre = []

for idx, dataframe in enumerate(df_array_pre):
    visual_stimulus_array = []

    #iter over every row 
    for _idx, row in dataframe.iterrows():
        data_str = row[config_visual_stimulus_variable]
        if isinstance(data_str, float)==True:
            continue
        data_str = data_str.strip()
        
        coordinates_str = data_str.split(" ")
        coordinates = []
       
        # iter over every coordinate pair x-y
        for coordinate_str in coordinates_str:
            try:
                coordinate = coordinate_str.split("-")
                coordinate = (int(coordinate[0]), int(coordinate[1]))
                coordinates.append(coordinate)
            except:
                print(coordinate_str)
            
        visual_stimulus_array.append(coordinates)
        
    visual_stimulus_data_matrix_pre.append(visual_stimulus_array)


In [10]:
visual_stimulus_data_matrix_post = []


for idx, dataframe in enumerate(df_array_post):
    visual_stimulus_array_post = []

    #iter over every row 
    for _idx, row in dataframe.iterrows():
        data_str_post = row[config_visual_stimulus_variable]
        data_str_post = data_str_post.strip()
        coordinates_str_post = data_str_post.split(" ")
        coordinates_post = []
       
        # iter over every coordinate pair x-y
        for coordinate_str_post in coordinates_str_post:
            try:
                coordinate_post = coordinate_str_post.split("-")
                coordinate_post = (int(coordinate_post[0]), int(coordinate_post[1]))
                coordinates_post.append(coordinate_post)
            except:
                print(coordinate_str_post)
            
            
        visual_stimulus_array_post.append(coordinates_post)
        
    visual_stimulus_data_matrix_post.append(visual_stimulus_array_post)

correct

In [11]:
visual_stimulus_data_matrix_pre_correct = []

for idx, dataframe in enumerate(df_array_pre_correct):
    visual_stimulus_array = []

    #iter over every row 
    for _idx, row in dataframe.iterrows():
        data_str = row[config_visual_stimulus_variable]
        data_str = data_str.strip()
        coordinates_str = data_str.split(" ")
        coordinates = []
       
        # iter over every coordinate pair x-y
        for coordinate_str in coordinates_str:
            try:
                coordinate = coordinate_str.split("-")
                coordinate = (int(coordinate[0]), int(coordinate[1]))
                coordinates.append(coordinate)
            except:
                print(coordinate_str)
            
            
        visual_stimulus_array.append(coordinates)
        
    visual_stimulus_data_matrix_pre_correct.append(visual_stimulus_array)

In [12]:
visual_stimulus_data_matrix_post_correct = []

for idx, dataframe in enumerate(df_array_post_correct):
    visual_stimulus_array = []

    #iter over every row 
    for _idx, row in dataframe.iterrows():
        data_str = row[config_visual_stimulus_variable]
        data_str = data_str.strip()
        coordinates_str = data_str.split(" ")
        coordinates = []
       
        # iter over every coordinate pair x-y
        for coordinate_str in coordinates_str:
            try:
                coordinate = coordinate_str.split("-")
                coordinate = (int(coordinate[0]), int(coordinate[1]))
                coordinates.append(coordinate)
            except:
                print(coordinate_str)
            
            
        visual_stimulus_array.append(coordinates)
        
    visual_stimulus_data_matrix_post_correct.append(visual_stimulus_array)

In [75]:
common_chars_stimulus_pre = visual_stimulus_data_matrix_pre[0]
common_chars_stimulus_post = visual_stimulus_data_matrix_post[0]

reverse_array_stimulus_pre = visual_stimulus_data_matrix_pre[3]
reverse_array_stimulus_post = visual_stimulus_data_matrix_post[3]

binary_search_stimulus_pre = visual_stimulus_data_matrix_pre[4]
binary_search_stimulus_post = visual_stimulus_data_matrix_post[4]

multiples_stimulus_pre = visual_stimulus_data_matrix_pre[5]
multiples_stimulus_post = visual_stimulus_data_matrix_post[5]

insertion_sort_stimulus_pre = visual_stimulus_data_matrix_pre[8]
insertion_sort_stimulus_post = visual_stimulus_data_matrix_post[8]


#0, 3, 4, 5, 8

wilcoxon_fixation_count_same = []

import scipy.stats as stats 




fixation_count_pre = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

fixation_count_post = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

for i in range(len(common_chars_stimulus_pre)):
    for x, y in common_chars_stimulus_pre[i]:
        if (x >= 45 and x <= 385) and ((y >= 169 and y <= 228) or (y >= 274 and y <= 348)):
            fixation_count_pre[0][i] += 1
            
for i in range(len(common_chars_stimulus_post)):
    for x, y in common_chars_stimulus_post[i]:
        #if (x >= 50 and x <= 136) and (y >= 169 and y <= 213):
        if (x >= 45 and x <= 391) and ((y >= 184 and y <= 243) or (y >= 288 and y <= 363)):
            fixation_count_post[0][i] += 1
            
for i in range(len(reverse_array_stimulus_pre)):
    for x, y in reverse_array_stimulus_pre[i]:
        if x >= 45 and (y >= 229 and y <= 303):
            fixation_count_pre[1][i] += 1
            
for i in range(len(reverse_array_stimulus_post)):
    for x, y in reverse_array_stimulus_post[i]:
        if x >= 45 and (y >= 205 and y <= 285):
            fixation_count_post[1][i] += 1
            
for i in range(len(binary_search_stimulus_pre)):
    for x, y in binary_search_stimulus_pre[i]:
        #if (x >= 40 and x <= 371) and ((y >= 169 and y <= 288) or (y >= 304 and y <= 408)):
        if (x >= 40 and x <= 371) and (y >= 169 and y <= 408):
            fixation_count_pre[2][i] += 1
            
for i in range(len(binary_search_stimulus_post)):
    for x, y in binary_search_stimulus_post[i]:
        #if (x >= 40 and x <= 369) and ((y >= 169 and y <= 288) or (y >= 304 and y <= 408)):
        if (x >= 40 and x <= 369) and (y >= 169 and y <= 408):
            fixation_count_post[2][i] += 1

for i in range(len(multiples_stimulus_pre)):
    for x, y in multiples_stimulus_pre[i]:
        if (x >= 45 and x <= 315) and (y >= 152 and y <= 234):
            fixation_count_pre[3][i] += 1
            
for i in range(len(multiples_stimulus_post)):
    for x, y in multiples_stimulus_post[i]:
        #if (x >= 50 and x <= 136) and (y >= 169 and y <= 213):
        if (x >= 45 and x <= 315) and (y >= 152 and y <= 234):
            fixation_count_post[3][i] += 1
            
            
for i in range(len(insertion_sort_stimulus_pre)):
    for x, y in insertion_sort_stimulus_pre[i]:
        if x >= 40 and (y >= 226 and y <= 360):
            fixation_count_pre[4][i] += 1
            
for i in range(len(insertion_sort_stimulus_post)):
    for x, y in insertion_sort_stimulus_post[i]:
        if x >= 40 and (y >= 226 and y <= 360):
            fixation_count_post[4][i] += 1
            
            
for f in fixation_count_pre:
    print(f)

print()
for f in fixation_count_post:
    print(f)

[18, 6, 3, 23, 17, 22, 25, 30, 50, 12, 9, 19, 33, 19, 5, 4, 21, 8, 36, 8, 13]
[6, 7, 10, 49, 3, 19, 10, 45, 7, 6, 4, 7, 30, 15, 4, 4, 10, 22, 9, 3, 11]
[4, 54, 27, 62, 14, 33, 27, 129, 147, 38, 37, 11, 90, 42, 24, 16, 41, 31, 35, 29]
[1, 6, 21, 12, 11, 6, 8, 31, 17, 19, 5, 8, 4, 10, 7, 1, 15, 5, 7, 9, 8]
[2, 3, 20, 53, 6, 33, 25, 17, 57, 6, 23, 4, 47, 9, 37, 4, 16, 27, 22, 8, 23]

[4, 10, 4, 11, 21, 15, 20, 19, 17, 11, 10, 9, 20, 6, 4, 5, 4, 9, 23, 7, 8]
[2, 2, 16, 9, 11, 18, 6, 12, 26, 15, 8, 6, 11, 6, 2, 2, 4, 9, 16, 0, 4]
[4, 21, 26, 2, 17, 45, 48, 57, 40, 33, 29, 10, 67, 65, 20, 21, 29, 14, 5, 19]
[2, 11, 11, 6, 12, 3, 6, 19, 4, 14, 8, 10, 12, 15, 7, 2, 17, 3, 6, 5, 6]
[3, 10, 8, 9, 6, 19, 23, 30, 60, 1, 6, 12, 60, 9, 20, 12, 22, 10, 5, 0, 26]


In [76]:
bar_df = pd.DataFrame([],columns=["Programmverständnisaufgaben","Klickanzahl Pretest", "Klickanzahl Posttest"])


pre_total = 0
post_total = 0

import statistics


print("S", statistics.stdev(fixation_count_pre[0]))
print("S", statistics.stdev(fixation_count_pre[1]))
print("S", statistics.stdev(fixation_count_pre[2]))
print("S", statistics.stdev(fixation_count_pre[3]))
print("S", statistics.stdev(fixation_count_pre[4]))

print("S", statistics.stdev(fixation_count_post[0]))
print("S", statistics.stdev(fixation_count_post[1]))
print("S", statistics.stdev(fixation_count_post[2]))
print("S", statistics.stdev(fixation_count_post[3]))
print("S", statistics.stdev(fixation_count_post[4]))

pre_list = []
post_list = []

post_list.extend(fixation_count_post[0])
post_list.extend(fixation_count_post[1])
post_list.extend(fixation_count_post[2])
post_list.extend(fixation_count_post[3])
post_list.extend(fixation_count_post[4])


pre_list.extend(fixation_count_pre[0])
pre_list.extend(fixation_count_pre[1])
pre_list.extend(fixation_count_pre[2])
pre_list.extend(fixation_count_pre[3])
pre_list.extend(fixation_count_pre[4])


print("STD", statistics.stdev(pre_list))
print("MEDIAN", statistics.median(pre_list))
print("STD", statistics.stdev(post_list))

print("MEDIAN", statistics.median(post_list))


for i in range(5):
    tmp_pre = 0
    tmp_post = 0
    
   
    for fixation in fixation_count_pre[i]:
        pre_total += fixation
        tmp_pre += fixation
    for fixation in fixation_count_post[i]:
        post_total += fixation
        tmp_post += fixation
        
        
    count = 21
    if i == 2:
        count = 20
    else:
        count = 21
    bar_df = bar_df.append(pd.DataFrame([["V" + str(i+1), tmp_pre/count, tmp_post/count]], 
                            columns=["Programmverständnisaufgaben", "Klickanzahl Pretest", "Klickanzahl Posttest"]))




print(pre_total/104, post_total/104)

display(bar_df)


bar_df = bar_df.append(pd.DataFrame([["Insgesamt", pre_total/104, post_total/104]], columns=["Programmverständnisaufgaben", "Klickanzahl Pretest", "Klickanzahl Posttest"]))
   
bar_df = bar_df.set_index('Programmverständnisaufgaben')
ax = bar_df.plot.bar(ylabel='Klickanzahl (AOI)', rot=0)
plt.tight_layout()
ax.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.savefig("results/behavioral/v_klickanzahl.pdf", bbox_inches='tight', pad_inches=0.01)

S 12.03031884151752
S 13.082339968355013
S 37.28478594759389
S 7.186627793869601
S 16.620698512626326
S 6.372933838844219
S 6.615278131862996
S 19.537817580002987
S 4.9961890238365445
S 16.58958365102288
STD 22.957399584195276
MEDIAN 15.0
STD 14.165299616274272
MEDIAN 10.0
21.21153846153846 14.653846153846153


Programmverständnisaufgaben  Klickanzahl Pretest  Klickanzahl Posttest
0                          V1            18.142857             11.285714
0                          V2            13.380952              8.809524
0                          V3            44.550000             28.600000
0                          V4            10.047619              8.523810
0                          V5            21.047619             16.714286

In [48]:
flat_list_pre = [item for sublist in fixation_count_pre for item in sublist]
flat_list_post = [item for sublist in fixation_count_post for item in sublist]

pre_box = pd.DataFrame(flat_list_pre, columns=["Klickanzahl (AOI)"])
pre_box["Test"] = "Pretest"


post_box = pd.DataFrame(flat_list_post, columns=["Klickanzahl (AOI)"])
post_box["Test"] = "Posttest"

zusammen = [pre_box, post_box]

aoi_plot = pd.concat(zusammen)

display(aoi_plot)

Klickanzahl (AOI)      Test
0                   18   Pretest
1                    6   Pretest
2                    3   Pretest
3                   23   Pretest
4                   17   Pretest
..                 ...       ...
99                  22  Posttest
100                 10  Posttest
101                  5  Posttest
102                  0  Posttest
103                 26  Posttest

[208 rows x 2 columns]

In [50]:
ax = sns.boxplot(x="Test", y="Klickanzahl (AOI)", data=aoi_plot)

ax.set_xlabel("");


ax.set_xticklabels(ax.get_xticklabels())
plt.ylim(0, 160)

plt.savefig("results/behavioral/v_klickanzahl_boxplot.pdf", bbox_inches='tight', pad_inches=0.01)

In [77]:


fixation_count_pre_list = [item for sublist in fixation_count_pre for item in sublist]
fixation_count_post_list = [item for sublist in fixation_count_post for item in sublist]


print(len(fixation_count_pre_list))
print(len(fixation_count_post_list))

def Average(lst):
    return sum(lst) / len(lst)
  
average_pre = Average(fixation_count_pre_list)
average_post = Average(fixation_count_post_list)

from cliffs_delta import cliffs_delta
from pingouin import wilcoxon
wilco = wilcoxon(fixation_count_pre_list, fixation_count_post_list)
cliff = cliffs_delta(fixation_count_pre_list, fixation_count_post_list)

print(wilco)


#round(wilcoxon_fixation_count_same[i]["p-val"][0], 5)
#round(cliffs_delta_fixation_count_same[i][0], 5)

df_fixation_count_same = pd.DataFrame([],
                                        columns=['Verständnisaufgaben', 'Pretest', 'Posttest', 'Wilcoxon', 'Cliffs-Delta'])
df_fixation_count_same = df_fixation_count_same.append({'Verständnisaufgaben': "V1 - V5",
                                                                            'Pretest': int(round(average_pre)),
                                                                            'Posttest': int(round(average_post)),
                                                                            'Wilcoxon': round(wilco["p-val"][0], 2),
                                                                            'Cliffs-Delta': round(cliff[0], 2)}, ignore_index=True)

display(df_fixation_count_same)
import os 

with open(os.path.join(os.getcwd(),"klickanzahl.tex"), "w"
) as tf:
    tf.write(df_fixation_count_same
             .to_latex(
             
             index=False,
             caption="Verständnisaufgaben: Durchschnittliche Klickanzahl",
                 escape=False,
                 column_format="ccccc",
             ))

104
104
           W-val alternative     p-val      RBC      CLES
Wilcoxon  1426.5   two-sided  0.000158  0.43505  0.592779


Verständnisaufgaben Pretest Posttest  Wilcoxon  Cliffs-Delta
0             V1 - V5      21       15       0.0          0.19

In [33]:
from cliffs_delta import cliffs_delta
from pingouin import wilcoxon


#fixation_count_df = pd.DataFrame([], columns=["V1-pre", "V1-post",
#                                              "V2-pre", "V2-post",
#                                              "V3-pre", "V3-post",
#                                              "V4-pre", "V4-post",
#                                              "V5-pre", "V5-post"])
#
#
#
#print(fixation_count_pre, fixation_count_post)
#
#for i in range(len(visual_stimulus_data_matrix_pre)):
#    if i == 0:
#        for i in range(len(visual_stimulus_data_matrix_pre[0])):
#            fixation_count_df.at[i, "V1-pre"] = len(visual_stimulus_data_matrix_pre[0][i])
#        for i in range(len(visual_stimulus_data_matrix_post[0])):
#            fixation_count_df.at[i, "V1-post"] = len(visual_stimulus_data_matrix_post[0][i])
#            
#    if i == 3:
#        for i in range(len(visual_stimulus_data_matrix_pre[3])):
#            fixation_count_df.at[i, "V2-pre"] = len(visual_stimulus_data_matrix_pre[3][i])
#        for i in range(len(visual_stimulus_data_matrix_post[3])):
#            fixation_count_df.at[i, "V2-post"] = len(visual_stimulus_data_matrix_post[3][i])
#            
#            
#    if i == 4:
#        for i in range(len(visual_stimulus_data_matrix_pre[4])):
#            fixation_count_df.at[i, "V3-pre"] = len(visual_stimulus_data_matrix_pre[4][i])
#        for i in range(len(visual_stimulus_data_matrix_post[4])):
#            fixation_count_df.at[i, "V3-post"] = len(visual_stimulus_data_matrix_post[4][i])
#            
#    if i == 5:
#        for i in range(len(visual_stimulus_data_matrix_pre[5])):
#            fixation_count_df.at[i, "V4-pre"] = len(visual_stimulus_data_matrix_pre[5][i])
#        for i in range(len(visual_stimulus_data_matrix_post[5])):
#            fixation_count_df.at[i, "V4-post"] = len(visual_stimulus_data_matrix_post[5][i])
#            
#    if i == 8:
#        for i in range(len(visual_stimulus_data_matrix_pre[8])):
#            fixation_count_df.at[i, "V5-pre"] = len(visual_stimulus_data_matrix_pre[8][i])
#        for i in range(len(visual_stimulus_data_matrix_post[8])):
#            fixation_count_df.at[i, "V5-post"] = len(visual_stimulus_data_matrix_post[8][i])
#
#
#df_fixation_count_same = pd.DataFrame([],
#                                        columns=['Verständnisaufgabe', 'Pretest', 'Posttest', 'Wilcoxon', 'Cliffs-Delta'])
#
#wilcoxon_fixation_count_same = []
#cliffs_delta_fixation_count_same = []
#
##tmp = wilcoxon(fixation_count_df["V1-pre"], fixation_count_df["V1-post"])
#
#display(fixation_count_df)
#
#names = ['V1-pre', 'V1-post',
#        'V2-pre', 'V2-post',
#        'V3-pre', 'V3-post',
#        'V4-pre', 'V4-post',
#        'V5-pre', 'V5-post']
#
#
#for name in names:
#    fixation_count_df[name] = pd.to_numeric(fixation_count_df[name])
#    
#wilcoxon_fixation_count_same.append(wilcoxon(fixation_count_df[names[0]], fixation_count_df[names[1]]))
#wilcoxon_fixation_count_same.append(wilcoxon(fixation_count_df[names[2]], fixation_count_df[names[3]]))
#wilcoxon_fixation_count_same.append(wilcoxon(fixation_count_df[names[4]], fixation_count_df[names[5]]))
#wilcoxon_fixation_count_same.append(wilcoxon(fixation_count_df[names[6]], fixation_count_df[names[7]]))
#wilcoxon_fixation_count_same.append(wilcoxon(fixation_count_df[names[8]], fixation_count_df[names[9]]))
#
#    
#cliffs_delta_fixation_count_same.append(cliffs_delta(fixation_count_df[names[0]], fixation_count_df[names[1]]))
#cliffs_delta_fixation_count_same.append(cliffs_delta(fixation_count_df[names[2]], fixation_count_df[names[3]]))
#cliffs_delta_fixation_count_same.append(cliffs_delta(fixation_count_df[names[4]], fixation_count_df[names[5]]))
#cliffs_delta_fixation_count_same.append(cliffs_delta(fixation_count_df[names[6]], fixation_count_df[names[7]]))
#cliffs_delta_fixation_count_same.append(cliffs_delta(fixation_count_df[names[8]], fixation_count_df[names[9]]))
#
#print(cliffs_delta_fixation_count_same[0][0])
#
#
#
#
#print(round(wilcoxon_fixation_count_same[0]["p-val"][0], 5))
#print(len(fixation_count_df))
#for i in range(5):
#    tmp_str = "V" + str(i+1)
#    df_fixation_count_same = df_fixation_count_same.append({'Verständnisaufgabe': "V" + str(i+1),
#                                                                            'Pretest': int(round(fixation_count_df[tmp_str+"-pre"].mean())),
#                                                                            'Posttest': int(round(fixation_count_df[tmp_str+"-post"].mean())),
#                                                                            'Wilcoxon': round(wilcoxon_fixation_count_same[i]["p-val"][0], 5),
#                                                                            'Cliffs-Delta': round(cliffs_delta_fixation_count_same[i][0], 5)}, ignore_index=True)
##import os 
#

#with open(os.path.join(os.getcwd(),"same_algorithm_fixation_count.tex"), "w"
#) as tf:
#    tf.write(df_fixation_count_same
#             .to_latex(
#             
#             index=False,
#             caption="Verständnisaufgaben: Durchschnittliche Klickanzahl",
#                 escape=False,
#                 column_format="ccccc",
#             ))

TypeError: 'int' object is not iterable

In [ ]:
display(df_fixation_count_same)

bar_df = pd.DataFrame([],columns=["Verständnisaufgaben","Klickanzahl Pretest", "Klickanzahl Posttest"])


pre_total = 0
post_total = 0

for i in range(5):
    bar_df = bar_df.append(pd.DataFrame([[df_fixation_count_same.iloc[i][0], df_fixation_count_same.iloc[i][1], df_fixation_count_same.iloc[i][2]]], 
                                    columns=["Verständnisaufgaben", "Klickanzahl Pretest", "Klickanzahl Posttest"]))
    pre_total += df_fixation_count_same.iloc[i][1]
    post_total += df_fixation_count_same.iloc[i][2]

pre_total /= 5
post_total /= 5

print(pre_total, post_total)

bar_df = bar_df.append(pd.DataFrame([["Insgesamt", pre_total, post_total]], columns=["Verständnisaufgaben", "Klickanzahl Pretest", "Klickanzahl Posttest"]))
    
bar_df = bar_df.set_index('Verständnisaufgaben')
ax = bar_df.plot.bar(ylabel='Klickanzahl (AOIs)', rot=0)
plt.tight_layout()
ax.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.savefig("results/behavioral/tmp.pdf", bbox_inches='tight', pad_inches=0.01)

In [183]:
import sonaion_analysis



In [186]:
from sonaion_analysis.utils.smallestenclosingcircle import make_circle


def create_fixxation_map(eye_x, eye_y, fixxation_classifier):
    points_array = []
    currently_fixxation = False
    current_points = []

    for idx, classifier in enumerate(fixxation_classifier):
        #if classifier == 1 and currently_fixxation == False:
        #    current_points = [(eye_x[idx], eye_y[idx])]
        #    currently_fixxation = True
        if classifier == 1:
            current_points.append((eye_x[idx], eye_y[idx]))
            points_array.append((current_points, True))
            current_points = []
        
        
        #elif classifier == 0 and currently_fixxation == True:
        #    points_array.append((current_points.copy(), True))
        #    current_points = []
        #    currently_fixxation = False
        #    points_array.append(([(eye_x[idx], eye_y[idx])], False))
        #else:
        #    points_array.append(([(eye_x[idx], eye_y[idx])], False))
    #circles = []
    #circle1 = plt.Circle((0, 0), 0.2, color='r')
    
    #for points, is_fixxation in points_array:
        #print(points, is_fixxation)
        #print(points[0][0])
        #circles.append(plt.Circle((points[0][0], points[0][1]), 30, color='r'))
    
    circles = [(make_circle(points), is_fixxation) for points, is_fixxation in points_array]
    circles = [((x, y), 30, is_fixxation) for ((x, y, radius), is_fixxation) in circles]
    
    return circles


def interpolate(start_color, end_color, current, max_value):
    t = current / float(max_value)
    r = (1.0 - t) * start_color[0] + t * end_color[0]
    g = (1.0 - t) * start_color[1] + t * end_color[1]
    b = (1.0 - t) * start_color[2] + t * end_color[2]
    return (r, g, b)

In [187]:
x_coordinates = []
y_coordinates = []

for x,y in visual_stimulus_data_matrix_post[1][5]:
    x_coordinates.append(x)
    y_coordinates.append(y)
    
fixations = [1] * len(x_coordinates)

img = image_array_post[1]
fixxation_map = create_fixxation_map(x_coordinates, y_coordinates, fixations)

left_color = (1.0, 0.0, 0.0)

fig, ax = plt.subplots()
x_val = [float(int(x)) for ((x, _), _, _) in fixxation_map]
y_val = [float(int(y)) for ((_, y), _, _) in fixxation_map]

for i in range(len(x_val) - 2):
    ax.plot(x_val[i : i + 2], y_val[i : i + 2], "-", color=interpolate(left_color, (0.0, 0.0, 0.0), i, len(x_val)), zorder=2, alpha=0.6)

filtered_x_val = [float(int(x)) for ((x, _), _, flag) in fixxation_map if flag]
filtered_y_val = [float(int(y)) for ((_, y), _, flag) in fixxation_map if flag]
ax.scatter(filtered_x_val, filtered_y_val, 30, c=[interpolate(left_color, (0.0, 0.0, 0.0), idx, len(x_val)) for idx, (_, _, flag) in enumerate(fixxation_map) if flag], zorder=3, alpha=0.6)
ax.imshow(img, zorder=1)
plt.show()

false

In [19]:
visual_stimulus_data_matrix_pre_false = []

for idx, dataframe in enumerate(df_array_pre_false):
    visual_stimulus_array = []

    #iter over every row 
    for _idx, row in dataframe.iterrows():
        data_str = row[config_visual_stimulus_variable]
        data_str = data_str.strip()
        coordinates_str = data_str.split(" ")
        coordinates = []
       
        # iter over every coordinate pair x-y
        for coordinate_str in coordinates_str:
            try:
                coordinate = coordinate_str.split("-")
                coordinate = (int(coordinate[0]), int(coordinate[1]))
                coordinates.append(coordinate)
            except:
                print(coordinate_str)
            
            
        visual_stimulus_array.append(coordinates)
        
    visual_stimulus_data_matrix_pre_false.append(visual_stimulus_array)

In [152]:
visual_stimulus_data_matrix_post_false = []

for idx, dataframe in enumerate(df_array_post_false):
    visual_stimulus_array = []

    #iter over every row 
    for _idx, row in dataframe.iterrows():
        data_str = row[config_visual_stimulus_variable]
        data_str = data_str.strip()
        coordinates_str = data_str.split(" ")
        coordinates = []
       
        # iter over every coordinate pair x-y
        for coordinate_str in coordinates_str:
            try:
                coordinate = coordinate_str.split("-")
                coordinate = (int(coordinate[0]), int(coordinate[1]))
                coordinates.append(coordinate)
            except:
                print(coordinate_str)
            
            
        visual_stimulus_array.append(coordinates)
        
    visual_stimulus_data_matrix_post_false.append(visual_stimulus_array)

<h4>Helper Functions</h4>

In [153]:
def save_images(image_array, folder, image_name):
    """
    :brief saves an array of images to a certain location incrementing the postfix by a number
    :param image_array:        array of images (np.ndarray)
    :param folder:     prefix of image/ folder location
    :param image_name: prefix for the image
    """
    
    Path(folder).mkdir(parents=True, exist_ok=True)
    
    prefix = folder + image_name
    
    #TODO create folders if there are none present
    for idx, data in enumerate(image_array):
        data = data*255
        data = np.uint8(data)
        im = Image.fromarray(data)
        im.save(prefix + str(idx) + '.png')
    
def is_in(value, tup):
    return tup[0] <= value <= tup[1]

def get_0_offset(number):
    i = 0
    number = int(number)
    while number != 0:
        number = int(number / 10)
        i = i + 1
    return i

In [31]:
test = visual_stimulus_data_matrix_pre[0][0]
print(test)

        
#im = rEYEker.draw_shape_heat_map(image_array_pre[0], test[0], click_setting, should_copy=True)
max_width = image_array_pre[0].shape[1]
max_height = image_array_pre[0].shape[0]

print(max_width, max_height)




[(98, 60), (105, 106), (356, 103), (98, 64), (322, 101), (146, 138), (383, 146), (133, 181), (171, 222), (139, 251), (138, 277), (120, 357), (120, 357), (141, 315), (126, 220), (138, 188), (187, 50), (159, 173), (297, 100), (187, 187), (155, 224), (165, 187), (195, 223), (196, 75), (150, 220), (121, 254), (156, 285), (179, 319), (179, 319), (144, 305), (212, 73), (180, 291), (141, 345)]
406 397


<h2>2. Create Single Heatmaps</h2>

create heatmaps false

In [22]:
heatmap_matrix_pre_false = []
print(algo_name_array_pre)
print("\tGoing to process datatable #" + str(idx) + " with " + str(len(visual_stimulus_data_matrix_pre_false)) + " datasets: ")

# iterate over all the datasets
for dataset_idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_pre_false):
    
    print("\t\tdataset #" + str(dataset_idx) + " (up to "+ str(len(stimulus_dataset)) + "): [", end='')
    heatmap_array = []

    # iterate over all the measurements of the dataset
    for visual_idx, stimulus_measurement in enumerate(stimulus_dataset):
        print(str(visual_idx), end=";")
        print()
        print("STIMULUS", stimulus_measurement)
        im = rEYEker.draw_shape_heat_map(image_array_pre[dataset_idx], stimulus_measurement, click_setting, should_copy=True)
        heatmap_array.append(im)
   
    print("]")
    heatmap_matrix_pre_false.append(heatmap_array)

['CommonChars' 'ContainsSubstring' 'CountVowels' 'ReverseArray'
 'BinarySearchStrings' 'Multiples' 'CrossSum' 'Swap' 'InsertionSort'
 'GreatestCommonDivisor']
	Going to process datatable #9 with 10 datasets: 
		dataset #0 (up to 3): [0;
STIMULUS [(182, 163), (182, 207), (122, 95), (102, 17), (110, 55), (147, 34), (147, 41), (140, 70), (137, 108), (226, 91), (205, 131), (195, 159), (186, 183), (144, 188), (142, 229), (127, 251), (118, 283), (129, 291), (156, 326), (144, 359), (172, 310), (180, 290), (179, 314), (181, 280), (166, 182), (181, 74), (202, 40), (199, 67), (191, 103), (186, 151), (186, 187), (185, 201), (177, 227), (167, 252), (153, 289), (178, 314), (144, 359), (204, 106), (195, 51), (191, 75), (202, 112), (202, 101), (198, 46), (191, 69), (185, 252), (166, 309), (196, 96), (195, 59), (192, 73)]
1;
STIMULUS [(220, 222), (220, 222), (211, 186), (80, 51), (108, 94), (104, 64), (96, 28), (77, 5), (128, 32), (164, 332), (150, 393), (122, 227), (171, 77), (171, 44), (166, 57), (1

KeyboardInterrupt: 

In [155]:
heatmap_matrix_post_false = []
print("\tGoing to process datatable #" + str(idx) + " with " + str(len(visual_stimulus_data_matrix_post_false)) + " datasets: ")

# iterate over all the datasets
for dataset_idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_post_false):
    
    print("\t\tdataset #" + str(dataset_idx) + " (up to "+ str(len(stimulus_dataset)) + "): [", end='')
    heatmap_array = []

    # iterate over all the measurements of the dataset
    for visual_idx, stimulus_measurement in enumerate(stimulus_dataset):
        print(str(visual_idx), end=";")
        
        im = rEYEker.draw_shape_heat_map(image_array_pre[dataset_idx], stimulus_measurement, click_setting, should_copy=True)
        heatmap_array.append(im)
   
    print("]")
    heatmap_matrix_post_false.append(heatmap_array)

	Going to process datatable #9 with 10 datasets: 
		dataset #0 (up to 3): [0;1;2;]
		dataset #1 (up to 0): []
		dataset #2 (up to 2): [0;1;]
		dataset #3 (up to 2): [0;1;]
		dataset #4 (up to 6): [0;1;2;3;4;5;]
		dataset #5 (up to 12): [0;1;2;3;4;5;6;7;8;9;10;11;]
		dataset #6 (up to 2): [0;1;]
		dataset #7 (up to 11): [0;1;2;3;4;5;6;7;8;9;10;]
		dataset #8 (up to 0): []
		dataset #9 (up to 3): [0;1;2;]


create heatmaps correct

In [156]:
heatmap_matrix_pre_correct = []
print("\tGoing to process datatable #" + str(idx) + " with " + str(len(visual_stimulus_data_matrix_pre_correct)) + " datasets: ")

# iterate over all the datasets
for dataset_idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_pre_correct):
    
    print("\t\tdataset #" + str(dataset_idx) + " (up to "+ str(len(stimulus_dataset)) + "): [", end='')
    heatmap_array = []

    # iterate over all the measurements of the dataset
    for visual_idx, stimulus_measurement in enumerate(stimulus_dataset):
        print(str(visual_idx), end=";")
        
        im = rEYEker.draw_shape_heat_map(image_array_pre[dataset_idx], stimulus_measurement, click_setting, should_copy=True)
        heatmap_array.append(im)
   
    print("]")
    heatmap_matrix_pre_correct.append(heatmap_array)

	Going to process datatable #9 with 10 datasets: 
		dataset #0 (up to 18): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;]
		dataset #1 (up to 18): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;]
		dataset #2 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #3 (up to 17): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;]
		dataset #4 (up to 15): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;]
		dataset #5 (up to 11): [0;1;2;3;4;5;6;7;8;9;10;]
		dataset #6 (up to 19): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;]
		dataset #7 (up to 19): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;]
		dataset #8 (up to 19): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;]
		dataset #9 (up to 19): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;]


In [157]:
heatmap_matrix_post_correct = []
print("\tGoing to process datatable #" + str(idx) + " with " + str(len(visual_stimulus_data_matrix_post_correct)) + " datasets: ")

# iterate over all the datasets
for dataset_idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_post_correct):
    
    print("\t\tdataset #" + str(dataset_idx) + " (up to "+ str(len(stimulus_dataset)) + "): [", end='')
    heatmap_array = []

    # iterate over all the measurements of the dataset
    for visual_idx, stimulus_measurement in enumerate(stimulus_dataset):
        print(str(visual_idx), end=";")
        
        im = rEYEker.draw_shape_heat_map(image_array_pre[dataset_idx], stimulus_measurement, click_setting, should_copy=True)
        heatmap_array.append(im)
   
    print("]")
    heatmap_matrix_post_correct.append(heatmap_array)

	Going to process datatable #9 with 10 datasets: 
		dataset #0 (up to 18): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;]
		dataset #1 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #2 (up to 19): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;]
		dataset #3 (up to 19): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;]
		dataset #4 (up to 14): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;]
		dataset #5 (up to 9): [0;1;2;3;4;5;6;7;8;]
		dataset #6 (up to 19): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;]
		dataset #7 (up to 10): [0;1;2;3;4;5;6;7;8;9;]
		dataset #8 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #9 (up to 18): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;]


create heatmaps

In [158]:
heatmap_matrix = []
print("\tGoing to process datatable #" + str(idx) + " with " + str(len(visual_stimulus_data_matrix_pre)) + " datasets: ")

# iterate over all the datasets
for dataset_idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_pre):
    
    print("\t\tdataset #" + str(dataset_idx) + " (up to "+ str(len(stimulus_dataset)) + "): [", end='')
    heatmap_array = []

    # iterate over all the measurements of the dataset
    for visual_idx, stimulus_measurement in enumerate(stimulus_dataset):
        print(str(visual_idx), end=";")
        
        im = rEYEker.draw_shape_heat_map(image_array_pre[dataset_idx], stimulus_measurement, click_setting, should_copy=True)
        heatmap_array.append(im)
   
    print("]")
    heatmap_matrix.append(heatmap_array)

	Going to process datatable #9 with 10 datasets: 
		dataset #0 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #1 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #2 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #3 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #4 (up to 20): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;]
		dataset #5 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #6 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #7 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #8 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #9 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]


In [159]:
heatmap_matrix_post = []
print("\tGoing to process datatable #" + str(idx) + " with " + str(len(visual_stimulus_data_matrix_post)) + " datasets: ")



#iterate over all the datasets
for dataset_idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_post):
    
    print("\t\tdataset #" + str(dataset_idx) + " (up to "+ str(len(stimulus_dataset)) + "): [", end='')
    heatmap_array_post = []
    
    # iterate over all the measurements of the dataset
    for visual_idx, stimulus_measurement in enumerate(stimulus_dataset):
        
        print(str(visual_idx), end=";")
       
        im = rEYEker.draw_shape_heat_map(image_array_post[dataset_idx], stimulus_measurement, click_setting, should_copy=True)
     
        heatmap_array_post.append(im)
   
    print("]")
    heatmap_matrix_post.append(heatmap_array)

	Going to process datatable #9 with 10 datasets: 
		dataset #0 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #1 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #2 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #3 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #4 (up to 20): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;]
		dataset #5 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #6 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #7 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #8 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #9 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]



save Heatmaps

In [160]:
for idx, heatmap_array in enumerate(heatmap_matrix):
    path = "./results/heatmaps/pretest/" + str(algo_name_array_pre[idx]) + "/"
    print("Writing to:" + path)
    save_images(heatmap_array, path, algo_name_array_pre[idx] + "_")


Writing to:./results/heatmaps/pretest/CommonChars/
Writing to:./results/heatmaps/pretest/ContainsSubstring/
Writing to:./results/heatmaps/pretest/CountVowels/
Writing to:./results/heatmaps/pretest/ReverseArray/
Writing to:./results/heatmaps/pretest/BinarySearchStrings/
Writing to:./results/heatmaps/pretest/Multiples/
Writing to:./results/heatmaps/pretest/CrossSum/
Writing to:./results/heatmaps/pretest/Swap/
Writing to:./results/heatmaps/pretest/InsertionSort/
Writing to:./results/heatmaps/pretest/GreatestCommonDivisor/


In [165]:
for idx, heatmap_array in enumerate(heatmap_matrix_pre_false):
    path = "./results/heatmaps/pretest/false/" + str(algo_name_array_pre[idx]) + "/"
    print("Writing to:" + path)
    save_images(heatmap_array, path, algo_name_array_pre[idx] + "_")

Writing to:./results/heatmaps/pretest/false/CommonChars/
Writing to:./results/heatmaps/pretest/false/ContainsSubstring/
Writing to:./results/heatmaps/pretest/false/CountVowels/
Writing to:./results/heatmaps/pretest/false/ReverseArray/
Writing to:./results/heatmaps/pretest/false/BinarySearchStrings/
Writing to:./results/heatmaps/pretest/false/Multiples/
Writing to:./results/heatmaps/pretest/false/CrossSum/
Writing to:./results/heatmaps/pretest/false/Swap/
Writing to:./results/heatmaps/pretest/false/InsertionSort/
Writing to:./results/heatmaps/pretest/false/GreatestCommonDivisor/


In [164]:
for idx, heatmap_array in enumerate(heatmap_matrix_pre_correct):
    path = "./results/heatmaps/pretest/correct/" + str(algo_name_array_pre[idx]) + "/"
    print("Writing to:" + path)
    save_images(heatmap_array, path, algo_name_array_pre[idx] + "_")

Writing to:./results/heatmaps/pretest/correct/CommonChars/
Writing to:./results/heatmaps/pretest/correct/ContainsSubstring/
Writing to:./results/heatmaps/pretest/correct/CountVowels/
Writing to:./results/heatmaps/pretest/correct/ReverseArray/
Writing to:./results/heatmaps/pretest/correct/BinarySearchStrings/
Writing to:./results/heatmaps/pretest/correct/Multiples/
Writing to:./results/heatmaps/pretest/correct/CrossSum/
Writing to:./results/heatmaps/pretest/correct/Swap/
Writing to:./results/heatmaps/pretest/correct/InsertionSort/
Writing to:./results/heatmaps/pretest/correct/GreatestCommonDivisor/


In [166]:
for idx, heatmap_array in enumerate(heatmap_matrix_post):
    path = "./results/heatmaps/posttest/" + str(algo_name_array_post[idx]) + "/"
    print("Writing to:" + path)
    save_images(heatmap_array, path, algo_name_array_post[idx] + "_")

Writing to:./results/heatmaps/posttest/CommonChars/
Writing to:./results/heatmaps/posttest/IsPalindrome/
Writing to:./results/heatmaps/posttest/ReverseString/
Writing to:./results/heatmaps/posttest/ReverseArray/
Writing to:./results/heatmaps/posttest/BinarySearchStrings/
Writing to:./results/heatmaps/posttest/Multiples/
Writing to:./results/heatmaps/posttest/Power/
Writing to:./results/heatmaps/posttest/GetMiddle/
Writing to:./results/heatmaps/posttest/InsertionSort/
Writing to:./results/heatmaps/posttest/SquareRoot/


In [167]:
for idx, heatmap_array in enumerate(heatmap_matrix_post_false):
    path = "./results/heatmaps/posttest/false/" + str(algo_name_array_pre[idx]) + "/"
    print("Writing to:" + path)
    save_images(heatmap_array, path, algo_name_array_pre[idx] + "_")

Writing to:./results/heatmaps/posttest/false/CommonChars/
Writing to:./results/heatmaps/posttest/false/ContainsSubstring/
Writing to:./results/heatmaps/posttest/false/CountVowels/
Writing to:./results/heatmaps/posttest/false/ReverseArray/
Writing to:./results/heatmaps/posttest/false/BinarySearchStrings/
Writing to:./results/heatmaps/posttest/false/Multiples/
Writing to:./results/heatmaps/posttest/false/CrossSum/
Writing to:./results/heatmaps/posttest/false/Swap/
Writing to:./results/heatmaps/posttest/false/InsertionSort/
Writing to:./results/heatmaps/posttest/false/GreatestCommonDivisor/


In [168]:
for idx, heatmap_array in enumerate(heatmap_matrix_post_correct):
    path = "./results/heatmaps/posttest/correct/" + str(algo_name_array_pre[idx]) + "/"
    print("Writing to:" + path)
    save_images(heatmap_array, path, algo_name_array_pre[idx] + "_")

Writing to:./results/heatmaps/posttest/correct/CommonChars/
Writing to:./results/heatmaps/posttest/correct/ContainsSubstring/
Writing to:./results/heatmaps/posttest/correct/CountVowels/
Writing to:./results/heatmaps/posttest/correct/ReverseArray/
Writing to:./results/heatmaps/posttest/correct/BinarySearchStrings/
Writing to:./results/heatmaps/posttest/correct/Multiples/
Writing to:./results/heatmaps/posttest/correct/CrossSum/
Writing to:./results/heatmaps/posttest/correct/Swap/
Writing to:./results/heatmaps/posttest/correct/InsertionSort/
Writing to:./results/heatmaps/posttest/correct/GreatestCommonDivisor/


<h2>3. Create Average Heatmaps</h2>

create pretest heatmaps

21


In [169]:
heatmap_array = []
mask_array = []
shape_array = []

# iterate over all the datasets
for idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_pre):
    print("#" + str(idx), end="")
    image = image_array_pre[idx]
    shape_array.append(image.shape)
    im, mask = rEYEker.draw_average_shape_heat_map_rel(image, stimulus_dataset, click_setting, 1.0, 0.0, None, should_copy=True)
    heatmap_array.append(im)
    mask_array.append(mask)


#0#1#2#3#4#5#6#7#8#9

In [170]:
for idx, heatmap in enumerate(heatmap_array):
    #path = "./results/" + str(algo_name_aray[algo_idx]) + "/heatmaps/average_heatmap/"
    path = "./results/averageHeatMaps/pretest/"
    print("Writing to:" + path)
    save_images([heatmap], path, algo_name_array_pre[idx] + "-Pretest")

Writing to:./results/averageHeatMaps/pretest/
Writing to:./results/averageHeatMaps/pretest/
Writing to:./results/averageHeatMaps/pretest/
Writing to:./results/averageHeatMaps/pretest/
Writing to:./results/averageHeatMaps/pretest/
Writing to:./results/averageHeatMaps/pretest/
Writing to:./results/averageHeatMaps/pretest/
Writing to:./results/averageHeatMaps/pretest/
Writing to:./results/averageHeatMaps/pretest/
Writing to:./results/averageHeatMaps/pretest/


In [171]:
heatmap_array = []
mask_array = []
shape_array = []

indizes = []

# iterate over all the datasets
for idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_pre_false):        
    print("#" + str(idx), end="")
    image = image_array_pre[idx]
    shape_array.append(image.shape)
    if len(stimulus_dataset) == 0:
        indizes.append(idx)
        continue
    im, mask = rEYEker.draw_average_shape_heat_map_rel(image, stimulus_dataset, click_setting, 1.0, 0.0, None, should_copy=True)
    heatmap_array.append(im)
    mask_array.append(mask)
    

#0#1#2#3#4#5#6#7#8#9

In [172]:
counter = 0

for idx, heatmap in enumerate(heatmap_array):
    if idx in indizes:
        counter += 1
    image_idx = idx + counter
    #path = "./results/" + str(algo_name_aray[algo_idx]) + "/heatmaps/average_heatmap/"
    path = "./results/averageHeatMaps/pretest/false/"
    #print("Writing to:" + path)
    save_images([heatmap], path, algo_name_array_pre[image_idx] + "-pretest-false")

In [173]:
heatmap_array = []
mask_array = []
shape_array = []

# iterate over all the datasets
for idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_pre_correct):
    print("#" + str(idx), end="")
    image = image_array_pre[idx]
    shape_array.append(image.shape)
    im, mask = rEYEker.draw_average_shape_heat_map_rel(image, stimulus_dataset, click_setting, 1.0, 0.0, None, should_copy=True)
    heatmap_array.append(im)
    mask_array.append(mask)
    


#0#1#2#3#4#5#6#7#8#9

In [174]:
for idx, heatmap in enumerate(heatmap_array):
    #path = "./results/" + str(algo_name_aray[algo_idx]) + "/heatmaps/average_heatmap/"
    path = "./results/averageHeatMaps/pretest/correct/"
    print("Writing to:" + path)
    save_images([heatmap], path, algo_name_array_pre[idx] + "-Pretest-correct")

Writing to:./results/averageHeatMaps/pretest/correct/
Writing to:./results/averageHeatMaps/pretest/correct/
Writing to:./results/averageHeatMaps/pretest/correct/
Writing to:./results/averageHeatMaps/pretest/correct/
Writing to:./results/averageHeatMaps/pretest/correct/
Writing to:./results/averageHeatMaps/pretest/correct/
Writing to:./results/averageHeatMaps/pretest/correct/
Writing to:./results/averageHeatMaps/pretest/correct/
Writing to:./results/averageHeatMaps/pretest/correct/
Writing to:./results/averageHeatMaps/pretest/correct/


save pretest heatmaps

create posttest heatmaps

In [175]:
heatmap_array = []
mask_array = []
shape_array = []

# iterate over all the datasets
for idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_post):
    print("#" + str(idx), end="")
    image = image_array_post[idx]
    shape_array.append(image.shape)
    im, mask = rEYEker.draw_average_shape_heat_map_rel(image, stimulus_dataset, click_setting, 1.0, 0.0, None, should_copy=True)
    heatmap_array.append(im)
    mask_array.append(mask)
    

#0#1#2#3#4#5#6#7#8#9

save posttest heatmaps

In [176]:
for idx, heatmap in enumerate(heatmap_array):
    #path = "./results/" + str(algo_name_aray[algo_idx]) + "/heatmaps/average_heatmap/"
    path = "./results/averageHeatMaps/posttest/"
    print(path)
    save_images([heatmap], path, algo_name_array_post[idx] + "-Posttest")

./results/averageHeatMaps/posttest/
./results/averageHeatMaps/posttest/
./results/averageHeatMaps/posttest/
./results/averageHeatMaps/posttest/
./results/averageHeatMaps/posttest/
./results/averageHeatMaps/posttest/
./results/averageHeatMaps/posttest/
./results/averageHeatMaps/posttest/
./results/averageHeatMaps/posttest/
./results/averageHeatMaps/posttest/


In [177]:
heatmap_array = []
mask_array = []
shape_array = []

indizes = []

# iterate over all the datasets
for idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_post_false):        
    print("#" + str(idx), end="")
    image = image_array_post[idx]
    shape_array.append(image.shape)
    if len(stimulus_dataset) == 0:
        indizes.append(idx)
        continue
    im, mask = rEYEker.draw_average_shape_heat_map_rel(image, stimulus_dataset, click_setting, 1.0, 0.0, None, should_copy=True)
    heatmap_array.append(im)
    mask_array.append(mask)

#0#1#2#3#4#5#6#7#8#9

In [178]:
counter = 0
image_idx = 0

for idx, heatmap in enumerate(heatmap_array):
    image_idx = idx + counter
    print(idx)
    #CommonChars 86.36363636363636
    #IsPalindrome 100.0
    #ReverseString 90.9090909090909
    #ReverseArray 90.9090909090909
    #BinarySearchStrings 72.72727272727273
    #Multiples 45.45454545454545
    #Power 90.9090909090909
    #GetMiddle 50.0
    #InsertionSort 100.0
    #SquareRoot 86.36363636363636
    #print(algo_name_array_post[idx], algo_name_array_post[image_idx])
    if image_idx in indizes:
        counter += 1
        image_idx += 1
    path = "./results/averageHeatMaps/posttest/false/"
    print("Writing to:" + path, algo_name_array_post[image_idx])
    save_images([heatmap], path, algo_name_array_post[image_idx] + "-posttest-false")

0
Writing to:./results/averageHeatMaps/posttest/false/ CommonChars
1
Writing to:./results/averageHeatMaps/posttest/false/ ReverseString
2
Writing to:./results/averageHeatMaps/posttest/false/ ReverseArray
3
Writing to:./results/averageHeatMaps/posttest/false/ BinarySearchStrings
4
Writing to:./results/averageHeatMaps/posttest/false/ Multiples
5
Writing to:./results/averageHeatMaps/posttest/false/ Power
6
Writing to:./results/averageHeatMaps/posttest/false/ GetMiddle
7
Writing to:./results/averageHeatMaps/posttest/false/ SquareRoot


In [179]:
heatmap_array = []
mask_array = []
shape_array = []

# iterate over all the datasets
for idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_post_correct):        
    print("#" + str(idx), end="")
    image = image_array_post[idx]
    shape_array.append(image.shape)
    im, mask = rEYEker.draw_average_shape_heat_map_rel(image, stimulus_dataset, click_setting, 1.0, 0.0, None, should_copy=True)
    heatmap_array.append(im)
    mask_array.append(mask)

#0#1#2#3#4#5#6#7#8#9

In [180]:
for idx, heatmap in enumerate(heatmap_array):
    #path = "./results/" + str(algo_name_aray[algo_idx]) + "/heatmaps/average_heatmap/"
    path = "./results/averageHeatMaps/posttest/correct/"
    print("Writing to:" + path)
    save_images([heatmap], path, algo_name_array_post[idx] + "-Posttest-correct")

Writing to:./results/averageHeatMaps/posttest/correct/
Writing to:./results/averageHeatMaps/posttest/correct/
Writing to:./results/averageHeatMaps/posttest/correct/
Writing to:./results/averageHeatMaps/posttest/correct/
Writing to:./results/averageHeatMaps/posttest/correct/
Writing to:./results/averageHeatMaps/posttest/correct/
Writing to:./results/averageHeatMaps/posttest/correct/
Writing to:./results/averageHeatMaps/posttest/correct/
Writing to:./results/averageHeatMaps/posttest/correct/
Writing to:./results/averageHeatMaps/posttest/correct/


<h2>4. Create Sequence diagramms</h2>

create sequence diagrams

In [134]:
sequence_diagrams_matrix = []
    
# iterate over all the datasets
for dataset_idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_pre):
    sequence_diagram_array = []
    print("\t\tdataset #" + str(dataset_idx) + " (up to "+ str(len(stimulus_dataset)) + "): [", end='')

    # iterate over all the measurements of the dataset
    for visual_idx, stimulus_measurement in enumerate(stimulus_dataset):
        print(str(visual_idx), end=";")
        im = image_array_pre[dataset_idx]
        try:
            im = rEYEker.draw_vertical_line_diagram(im, stimulus_measurement, should_copy=True)
            sequence_diagram_array.append(im)
            
        except:
            #TODO
            sequence_diagram_array.append(im.copy())
            print("W.I.P.:", end='')
            print("to many clicks for dataset " + str(dataset_idx) + " datset " + str(visual_idx))
            
    print("]")
    sequence_diagrams_matrix.append(sequence_diagram_array)

		dataset #0 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #1 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #2 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #3 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #4 (up to 20): [0;1;2;3;4;5;6;7;W.I.P.:to many clicks for dataset 4 datset 7
8;W.I.P.:to many clicks for dataset 4 datset 8
9;10;11;12;13;14;15;16;17;18;19;]
		dataset #5 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #6 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #7 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #8 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #9 (up to 21): [0;1;2;3;4;5;6;7;8;W.I.P.:to many clicks for dataset 9 datset 8
9;10;11;12;13;14;15;16;17;18;19;20;]


save sequence diagrams

In [136]:
for idx, sequence_diagram_array in enumerate(sequence_diagrams_matrix):
    path = "./results/sequence_diagrams/pretest/" +  str(algo_name_array_pre[idx]) + "/"
    print("Writing to:" + path)
    save_images(sequence_diagram_array, path, algo_name_array_pre[idx])

Writing to:./results/sequence_diagrams/pretest/CommonChars/
Writing to:./results/sequence_diagrams/pretest/ContainsSubstring/
Writing to:./results/sequence_diagrams/pretest/CountVowels/
Writing to:./results/sequence_diagrams/pretest/ReverseArray/
Writing to:./results/sequence_diagrams/pretest/BinarySearchStrings/
Writing to:./results/sequence_diagrams/pretest/Multiples/
Writing to:./results/sequence_diagrams/pretest/CrossSum/
Writing to:./results/sequence_diagrams/pretest/Swap/
Writing to:./results/sequence_diagrams/pretest/InsertionSort/
Writing to:./results/sequence_diagrams/pretest/GreatestCommonDivisor/


In [138]:
sequence_diagrams_matrix = []
    
# iterate over all the datasets
for dataset_idx, stimulus_dataset in enumerate(visual_stimulus_data_matrix_post):
    sequence_diagram_array = []
    print("\t\tdataset #" + str(dataset_idx) + " (up to "+ str(len(stimulus_dataset)) + "): [", end='')

    # iterate over all the measurements of the dataset
    for visual_idx, stimulus_measurement in enumerate(stimulus_dataset):
        print(str(visual_idx), end=";")
        im = image_array_pre[dataset_idx]
        try:
            im = rEYEker.draw_vertical_line_diagram(im, stimulus_measurement, should_copy=True)
            sequence_diagram_array.append(im)
            
        except:
            #TODO
            sequence_diagram_array.append(im.copy())
            #print("W.I.P.:", end='')
            print("to many clicks for dataset " + str(dataset_idx) + " dataset " + str(visual_idx))
            
    print("]")
    sequence_diagrams_matrix.append(sequence_diagram_array)

		dataset #0 (up to 21): [0;1;2;3;4;to many clicks for dataset 0 dataset 4
5;6;7;to many clicks for dataset 0 dataset 7
8;9;10;11;12;13;14;to many clicks for dataset 0 dataset 14
15;16;17;18;19;to many clicks for dataset 0 dataset 19
20;to many clicks for dataset 0 dataset 20
]
		dataset #1 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #2 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #3 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #4 (up to 20): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;]
		dataset #5 (up to 21): [0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #6 (up to 21): [0;1;to many clicks for dataset 6 dataset 1
2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;]
		dataset #7 (up to 21): [0;to many clicks for dataset 7 dataset 0
1;to many clicks for dataset 7 dataset 1
2;to many clicks for dataset 7 dataset 2
3;to many clicks for dataset 7 dataset 3
4;to many